Metric Learning -- learning a good distance metric for KNN

Notes for LMNN (Large Margin Nearest Neighbors): Weinberger, K. Q., Saul, L. K. *Distance Metric Learning for Large Margin Nearest Neighbor Classification. JMLR 2009*.

Often, when we want to use kNN to classification, it is possible that different dimensions of the data have different scale and thus a traditional Euclidean distance metric is not appropriate. It is desirable to give different weights to different dimensions in determining the distance b/t two points. It would be nice if we can automatically figure that distance metric out (i.e. learn a good distance metric) from the present training data set.

LMNN is an algorithm whose motivation is, given the number of nearest neighbors k, learn the Mahalanobis matrix (a general way to represent the distance measure, meaning any distance metric d(x,y) can be represented by d(x,y) = (x-y)^T M (x-y)) that maximizes the kNN accuracy.

LMNN can be interpresented in two ways:
    1. Linear transformation: LMNN can be seen as a linear transformation L on the original data points X. Then it uses a simple Euclidean measure to perform a kNN classification on the transformed data points LX.
    2. Learn a Mahalanobis distance: LMNN can also be seen as a procedure to learn a Mahalanobis distance M so that we compute the distance b/t any two points by d(x,y) = (x-y)^T M (x-y). Then we do the a kNN classification with this new distance metric. 

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

from pylmnn import LargeMarginNearestNeighbor as LMNN

Let's first load the data and split it into training set (60%) and test set (40%)

In [2]:
features_file = 'AwA2-features/Animals_with_Attributes2/Features/ResNet101/AwA2-features.txt'
labels_file = 'AwA2-features/Animals_with_Attributes2/Features/ResNet101/AwA2-labels.txt'

# There is in total 37322 images of 50 classes. Each image is represented as a 2048 dimensional feature
features = np.loadtxt(features_file) # shape (37322, 2048)
labels = np.loadtxt(labels_file) # shape (37322, )

# Split each and all classes into training set (60%) and test set (40%)
# set random_state to an int for reproducibility
X_train, X_test, Y_train, Y_test = train_test_split(
    features, labels, train_size=0.6, test_size=0.4, random_state=0, stratify=labels)


Train a LMNN and use the trained metric to transform the data, and then build a kNN classifier out of the transformed data.

In [8]:
# Set up the hyperparameters
k_train, k_test, n_components, max_iter = 7, 7, X_train.shape[1], 400

# Instantiate the metric learner
lmnn = LMNN(n_neighbors=k_train, max_iter=max_iter, n_components=n_components)

# Train the metric learner
lmnn.fit(X_train, Y_train)

print(str(max_iter) + ' iterations of LMNN training is finished!')

# Fit the nearest neighbors classifier
clf = KNeighborsClassifier(n_neighbors=k_test)
clf.fit(lmnn.transform(X_train), Y_train)

# Compute the k-nearest neighbor test accuracy after applying the learned transformation
accuracy = clf.score(lmnn.transform(X_test), Y_test)
print('Accuracy (with learned distance metric) = ' + str(accuracy))

400 iterations of LMNN training is finished!
Accuracy (with learned distance metric) = 0.9314756514167057


5 iterations -->  0.9082322995512091

50 iterations --> 0.9210931743586308

100 iterations --> 0.9182798579945073

200 iteartions --> 0.9311407328019291

400 iterations --> 0.9314756514167057

